In [1]:
%pip install tensorflow opencv-python matplotlib

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ba/7c/b971f2485155917ecdcebb210e021e36a6b65457394590be01cc61515310/tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/fb/c4/f574ba6f04e6d7bf8c38d23e7a52389566dd7631fee0bcdd79ea07ef2dbf/opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/da/88/513a104dc2b3e9c1395598480466c55d2b1358e05c7198764670c4b1ddd0/matplotlib-3.7.3-cp310-cp310-win_amd64.whl.metadata
  Using cached matplotlib-3.7.3-cp310-cp310-win_amd64.whl.metadata (5.8 kB)
Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl (1.9 kB)
Using cached opencv_python-4.8.0.76-cp37-abi3-win_am


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Standard dependencies
import cv2
import os
import random
import numpy as np # For re-shaping arrays
from matplotlib import pyplot as plt # Visualise images

# Tensorflow dependencies
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [6]:
# To avoid running out of memory, we restrict the GPU memory growth aka
# how many resources the model is consuming at any given time
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    # Just in case the device we are using has more than one gpu, we are 
    # making sure to restrict the usage of ALL of them
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# Create files that will hold the anchor, positive and negative images:
# Anchor: The image we imput
# Positive: Images that match the anchor
# Negative: Images that are different from the anchor

os.makedirs(os.path.join("data","anchor"))
os.makedirs(os.path.join("data","positive"))
os.makedirs(os.path.join("data","negative"))


In [1]:
# Uncompress the tar file that contains the images in the Wild Dataface
!tar -xf lfw.tgz

In [7]:
# We take the images that we downloaded and place in the negative folder
# (these images will be used so that the machine can understand that the
# person whose image we are providing isn't the same as any of the ones in
# the negative folder)

# Go through all the directories in the lfw folder
for directory in os.listdir('lfw'):
    # Find all the images in said directory
    for file in os.listdir(os.path.join('lfw',directory)):
        # Replace the path of that image with the path of the negative folder
        # (aka place the image in the negative folder)
        previous_path = os.path.join('lfw', directory, file)
        new_path = os.path.join(os.path.join("data","negative"), file)
        os.replace(previous_path, new_path)

In [5]:
# Importing this  so that the images we save  all have different names
import uuid

# Now we will get the images we will require for the anchor and positive files

# Connect to the webcam
capture = cv2.VideoCapture(0) # Keep in mind this number might vary slightly
                              # so try out a few other numbers like 2, 4, 5 etc in case there is a problem
while (capture.isOpened()):
    return_value, frame = capture.read()

    # Keeping in mind that the images in the negative folder have a resolution of 250x250
    # we need out frames (aka the images we will capture) to be 250x250 as well
    frame = frame[120:370, 200:450, :]

    # Show the camera feed
    cv2.imshow("Images", frame)

    # Add image to anchor if 'a' is pressed
    if (cv2.waitKey(1) & 0XFF == ord('a')):
        # Create the unique name and save the image
        name = os.path.join(os.path.join("data","anchor"), '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(name, frame)
    
    # Add image to positive if 'p' is pressed
    if (cv2.waitKey(1) & 0XFF == ord('p')):
        # Create the unique name and save the image
        name = os.path.join(os.path.join("data","positive"), '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(name, frame)

    # Break by pressing he 'q' key
    if (cv2.waitKey(1) & 0XFF == ord('q')):
        break

# Release webcam
capture.release()
# Close the camera feed window
cv2.destroyAllWindows()